# PA1.1 Text Generation using Shannon Visualization Method

### Introduction

In this notebook, you will be generating text using the Shannon Visualization method.

An n-gram is a contiguous sequence of n words. For example "Machine" is a unigram, "Machine Learning" is a bigram and "Machine Learning PA1" is a trigram. In language modeling, n-gram models are probabilistic models of text that use word dependencies and context to predict the likelihood of occurence of an n-gram, i.e. predicting the nth word in an n-gram based on the previous n-1 words. One use of the predictions made by such a model is text generation. In this part, you will be generating text using the Shannon Visualization Method.

For additional details of the working of n-gram models and shannon visualization method, you can also consult [Chapter 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) of the SLP3 book as reference.

### Instructions

- Follow along with the notebook, filling out the necessary code where instructed.

- <span style="color: red;">Read the Submission Instructions, Plagiarism Policy, and Late Days Policy in the attached PDF.</span>

- <span style="color: red;">Make sure to run all cells for credit.</span>

- <span style="color: red;">Do not remove any pre-written code.</span>

- <span style="color: red;">You must attempt all parts.</span>

For this notebook, in addition to standard libraries i.e. `numpy`, `pandas`, `regex`, `matplotlib` and `scipy`, you **can** use [UrduHack](https://github.com/urduhack/urduhack) for tokenization, and [NLTK](https://www.nltk.org/) for training your n-grams. However, no other machine learning toolkits or libraries are allowed.

In [1]:
# import all required libraries here
import numpy as np 
import pandas as pd 
import regex as re 
import matplotlib.pyplot as mpl 
import scipy as sp 
import urduhack as uh 
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import random

### Dataset

You will be using the Urdu short stories by Patras Bukhari given in the folder `Urdu Short Stories` in the attached zip file for the purposes of this part of the assignment. This contains 6 stories of varying lengths which will serve as inputs for your n-gram model. 

You're required to implement an n-gram model that uses the given stories to generate Urdu text that mimics the input stories.

## Loading and Preprocessing the Dataset

Read in the short story files given and tokenize the text to be preprocessed.

In [2]:
# code here
'''
Function to read all the text stored in the files inside a folder
'''
def read_text_files(path, start_char, stop_char): # path to the folder
    all_text = ""
    for filename in os.listdir(path=path):
        file_path = os.path.join(path, filename)
        if os.path.isfile(file_path) and filename.endswith(".txt"):
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
                file_content = file_content.replace(".", f".{stop_char}")
                file_content = file_content.replace("\n", f"{stop_char}\n{start_char}")
                all_text += f"{start_char}{file_content}{stop_char}"
    return all_text


# print(os.listdir("DataP1/DataP1"))
text_folder_path = "DataP1/DataP1"
start_char = "s"
stop_char = "e"
text = read_text_files(text_folder_path, start_char, stop_char)


In [3]:
print(text)

s’’سینما کا عشق‘‘ عنوان تو عجب ہوس خیز ہے، لیکن افسوس کہ اس مضمون سے آپ کی تمام توقعات مجروح ہوں گی۔ کیونکہ مجھے تو اس مضمون میں کچھ دل کے داغ دکھانے مقصود ہیں۔ e
se
sاس سے آپ یہ نہ سمجھئے کہ مجھے فلموں سے دلچسپی نہیں، یا سینما کی موسیقی اور تاریکی میں جو ارمان انگیزی ہے میں اس کا قائل نہیں۔ میں تو سینما کے معاملے میں اوائل عمر ہی سے بزرگوں کا مورد عتاب رہ چکا ہوں، لیکن آج کل ہمارے دوست مرزا صاحب کی مہربانیوں کی طفیل سینما گویا میری ایک دکھتی رگ بن کر رہ گیا ہے۔ جہاں اس کا نام سن پاتا ہوں بعض دردانگیز واقعات کی یاد تازہ ہوجاتی ہے، جس سے رفتہ رفتہ میری فطرت ہی کج بیں بن گئی ہے۔ e
se
sاول تو خدا کے فضل سے ہم سینما کبھی وقت پر نہیں پہنچ سکے۔ اس میں میری سستی کو ذرا دخل نہیں۔ یہ سب قصور ہمارے دوست مرزا صاحب کا ہے جو کہنے کو تو ہمارے دوست ہیں لیکن خدا شاہد ہے، ان کی دوستی سے جو نقصان ہمیں پہنچے ہیں کسی دشمن کے قبضئہ قدرت سے بھی باہر ہوں گے۔ جب سینما جانے کا ارادہ ہو، ہفتہ بھر پہلے سے انہیں کہہ رکھتا ہوں کہ کیوں بھئی مرزا اگلی جمعرات سینما چلو گے نا! میری مراد یہ ہوتی ہے کہ وہ پہلے سے تیار ر

In [4]:
# normalizing the text before tokenizing
normalized_text = uh.normalize(text)
# tokenize (word tokenization) the normalized text
tokens = word_tokenize(normalized_text)
print(tokens)

['s', '’', '’', 'سینما', 'کا', 'عشق', '‘', '‘', 'عنوان', 'تو', 'عجب', 'ہوس', 'خیز', 'ہے،', 'لیکن', 'افسوس', 'کہ', 'اس', 'مضمون', 'سے', 'آپ', 'کی', 'تمام', 'توقعات', 'مجروح', 'ہوں', 'گی۔', 'کیونکہ', 'مجھے', 'تو', 'اس', 'مضمون', 'میں', 'کچھ', 'دل', 'کے', 'داغ', 'دکھانے', 'مقصود', 'ہیں۔', 'e', 'se', 's', 'اس', 'سے', 'آپ', 'یہ', 'نہ', 'سمجھئے', 'کہ', 'مجھے', 'فلموں', 'سے', 'دلچسپی', 'نہیں،', 'یا', 'سینما', 'کی', 'موسیقی', 'اور', 'تاریکی', 'میں', 'جو', 'ارمان', 'انگیزی', 'ہے', 'میں', 'اس', 'کا', 'قائل', 'نہیں۔', 'میں', 'تو', 'سینما', 'کے', 'معاملے', 'میں', 'اوائل', 'عمر', 'ہی', 'سے', 'بزرگوں', 'کا', 'مورد', 'عتاب', 'رہ', 'چکا', 'ہوں،', 'لیکن', 'آج', 'کل', 'ہمارے', 'دوست', 'مرزا', 'صاحب', 'کی', 'مہربانیوں', 'کی', 'طفیل', 'سینما', 'گویا', 'میری', 'ایک', 'دکھتی', 'رگ', 'بن', 'کر', 'رہ', 'گیا', 'ہے۔', 'جہاں', 'اس', 'کا', 'نام', 'سن', 'پاتا', 'ہوں', 'بعض', 'دردانگیز', 'واقعات', 'کی', 'یاد', 'تازہ', 'ہوجاتی', 'ہے،', 'جس', 'سے', 'رفتہ', 'رفتہ', 'میری', 'فطرت', 'ہی', 'کج', 'بیں', 'بن', 'گئی', 'ہے۔'

Preprocess the tokenized data. Go through the data and use your own discretion to decide on what kind of pre-processing might be required.

In [5]:
# code here
'''
alrady done using the urduhack normalize function
'''

'\nalrady done using the urduhack normalize function\n'

## Creating Unigrams

Generate a list of unigrams. Print the first 10 unigrams obtained.

In [6]:
# code here
ten_unigrams = random.sample(tokens, 10)
for word in ten_unigrams:
    print(word, end = ' ')

شمع ایسی بیٹھ کس ہو کے عافیت بھی جائیں ‘ 

Find the probabilities for each unique unigram. (Refer to the Shannon Visualization Method that we studied in class.)

In [7]:
# code here
word_count_dict = Counter(tokens)
token_count = len(tokens)
word_probabilities = {}
for word, count in word_count_dict.items():
    word_probabilities[word] = count/token_count
sorted_word_prob = dict(sorted(word_probabilities.items(), key=lambda item: item[1]))
# print(word_probabilities)
print(word_count_dict)
print(sorted_word_prob)


Counter({'میں': 480, 'کے': 453, 'اور': 352, 'کی': 336, 'سے': 305, 's': 272, 'e': 272, '’': 264, 'کہ': 263, 'اس': 247, 'کا': 242, 'se': 238, '‘': 225, 'تو': 218, 'کو': 216, 'پر': 161, 'ایک': 156, 'ہے۔': 156, 'نے': 150, 'یہ': 148, 'ہے': 145, 'کر': 144, 'بھی': 122, 'نہ': 117, 'ہم': 109, 'ان': 109, 'آپ': 108, 'لیکن': 103, 'ہیں۔': 95, 'ہو': 91, 'کیا': 84, 'ہی': 82, 'وہ': 82, 'نہیں': 81, 'ہیں': 70, 'جو': 69, 'ہوں': 65, 'پھر': 63, 'کوئی': 58, 'یا': 57, 'اب': 55, 'کسی': 47, 'بعد': 47, 'ہر': 47, 'ہوں۔': 45, 'بہت': 44, 'لاہور': 43, 'کچھ': 42, 'اپنے': 42, 'تھا': 42, 'تک': 41, 'لیے': 41, '!': 40, 'جب': 39, 'ہے،': 38, 'وقت': 38, 'ساتھ': 37, 'اپنی': 36, 'رہے': 36, 'ہوا': 36, 'صرف': 34, 'کرتے': 34, 'طرح': 34, 'سب': 33, 'لئے': 33, 'سال': 33, 'دل': 32, 'جس': 32, 'تھا۔': 32, 'ہاسٹل': 32, 'مجھے': 31, 'ہیں،': 31, 'دو': 31, 'بجے': 30, 'پاس': 30, 'ہمارے': 29, 'شروع': 29, 'ہوتا': 29, 'ہمیں': 28, 'میری': 27, 'بعض': 27, 'ضرور': 27, 'دن': 26, 'دیا': 26, 'جاتا': 26, 'ہونے': 26, 'صاحب': 25, 'بات': 25, 'گیا': 24, 

## Creating Bigrams

Generate a list of bigrams. Print the first 10 bigrams obtained.

In [8]:
# code here
corpus_bigrams = list(nltk.bigrams(tokens))
print(corpus_bigrams[:10])

[('s', '’'), ('’', '’'), ('’', 'سینما'), ('سینما', 'کا'), ('کا', 'عشق'), ('عشق', '‘'), ('‘', '‘'), ('‘', 'عنوان'), ('عنوان', 'تو'), ('تو', 'عجب')]


Find the probabilities for each unique bigram. 

In [9]:
bigram_count_dic = {}
for tupl in corpus_bigrams:
    word1 = tupl[0]
    word2 = tupl[1]
    if word1 not in bigram_count_dic:
        bigram_count_dic[word1] = {}
    if word2 not in bigram_count_dic[word1]:
        bigram_count_dic[word1][word2] = 1
    else:
        bigram_count_dic[word1][word2] += 1 


In [10]:
print(bigram_count_dic)

{'s': {'’': 37, 'اس': 11, 'اول': 2, 'ان': 4, 'میری': 1, 'مرزا': 3, 'خیر': 1, 'وہاں': 1, 'آخرکار': 1, 'آنکھیں': 1, 'اگر': 3, 'تمام': 1, 'ٹکٹ': 1, 'تھوڑی': 1, 'اب': 9, 'میں': 19, 'پیچھے': 1, 'لیکن': 9, 'اے': 1, 'چند': 1, 'یہ': 5, 'ببانگ': 1, 'اور': 3, 'پچھلے': 2, 'مسلمان': 1, 'لاہور': 7, 'آپ': 2, 'ہمارے': 1, 'فی': 1, 'باقی': 2, 'کسی': 2, 'مجھ': 1, 'ایسے': 1, 'کیا': 3, 'فنون': 1, 'ہیں': 1, 'پابستگیٴ': 1, 'ایک': 4, 'فرحتے': 1, 'روز': 1, 'منزل': 1, 'کوئی': 1, 'خدا': 1, 'جب': 6, 'تھرڈ': 1, 'بہر': 1, 'ہمارا': 1, 'چنانچے': 1, 'ضرورت': 1, 'سنیما': 1, 'چنانچہ': 1, 'اگلی': 1, 'کالج': 2, 'بناداں': 1, 'کہ': 2, 'کہنے': 7, 'ہر': 2, 'شروع': 1, 'اتفاق': 1, 'ہماری': 1, 'فارسی': 1, 'جن': 1, '(': 6, 'گویا': 1, 'اتنی': 1, 'آخری': 2, 'ساتھ': 1, 'والد': 1, 'اپنے': 1, 'تمہید': 1, 'محل': 1, 'حدود': 1, 'کہتے': 1, 'آب': 1, 'بہم': 1, 'نظام': 1, 'ذرائع': 1, 'جو': 1, 'بعض': 2, 'اصلی': 1, 'قابل': 1, 'رفتہ': 2, 'e': 2, 'صنعت': 1, 'اشتہاروں': 1, 'پیداوار': 1, 'رخ': 1, 'ادھر': 1, 'شمعیں': 1, 'تیسری': 1, 'چوتھی': 1, 'طب

In [11]:
'''
Now to calculate the probabilities of each bigram we take each tuple from the original list and check the occurences of second word given first word

'''
bigram_probabilities = {}
for tupl in corpus_bigrams:
    word1 = tupl[0]
    word2 = tupl[1]
    b_count = bigram_count_dic[word1][word2]
    total_sum = sum(bigram_count_dic[word1].values())
    

    bigram_probabilities[tupl] = b_count / total_sum

In [12]:
print(bigram_probabilities)

{('s', '’'): 0.13602941176470587, ('’', '’'): 0.5, ('’', 'سینما'): 0.003787878787878788, ('سینما', 'کا'): 0.07692307692307693, ('کا', 'عشق'): 0.004132231404958678, ('عشق', '‘'): 0.25, ('‘', '‘'): 0.49777777777777776, ('‘', 'عنوان'): 0.0044444444444444444, ('عنوان', 'تو'): 1.0, ('تو', 'عجب'): 0.0045871559633027525, ('عجب', 'ہوس'): 0.2, ('ہوس', 'خیز'): 0.5, ('خیز', 'ہے،'): 1.0, ('ہے،', 'لیکن'): 0.18421052631578946, ('لیکن', 'افسوس'): 0.009708737864077669, ('افسوس', 'کہ'): 1.0, ('کہ', 'اس'): 0.03802281368821293, ('اس', 'مضمون'): 0.020242914979757085, ('مضمون', 'سے'): 0.1111111111111111, ('سے', 'آپ'): 0.022950819672131147, ('آپ', 'کی'): 0.08333333333333333, ('کی', 'تمام'): 0.002976190476190476, ('تمام', 'توقعات'): 0.07692307692307693, ('توقعات', 'مجروح'): 1.0, ('مجروح', 'ہوں'): 0.5, ('ہوں', 'گی۔'): 0.015384615384615385, ('گی۔', 'کیونکہ'): 0.14285714285714285, ('کیونکہ', 'مجھے'): 0.1111111111111111, ('مجھے', 'تو'): 0.03225806451612903, ('تو', 'اس'): 0.03211009174311927, ('مضمون', 'میں'): 0.

## Generating Text using the Shannon Visualization Method

Generate a paragraph with ten sentences. Use the Shannon visualization method that we studied in class.

In [20]:
# code here
MAX_SENTENCE_LENGTH = 20
def generate_sentence(bigram_count_dic, tokens, max_sentence_length=MAX_SENTENCE_LENGTH):
    # first word should be sampled from tokens
    sentence_list = []
    word1 = 'e'
    while (word1 in ['se', 's', 'e']):
        word1 = random.sample(tokens, 1)
    word1 = word1[0]
    sentence_list.append(word1)

    for i in range(max_sentence_length):
        if word1 == 'e':
            break
        next_possible_word_dic = bigram_count_dic[word1]
        word_list = [key for key, value in next_possible_word_dic.items() for i in range(value)]
        next_word = random.sample(word_list, 1)
        if (next_word not in ['se', 's'] ):
            next_word = next_word[0]
            sentence_list.append(next_word)
            word1 = next_word
    return " ".join(sentence_list)


In [14]:
sent = generate_sentence(bigram_count_dic=bigram_count_dic, tokens=tokens)

In [21]:
paragraph = []
for i in range(10):
    sentence = generate_sentence(bigram_count_dic=bigram_count_dic, tokens=tokens)
    paragraph.append(sentence)
    print(sentence)
    

صراحی پر مس سلوچنا اور دنیا میں عام ضروریات کے مالک ہیں، تتلا کے ذرائع آمد و خوض کے داغ دکھانے
نہایت خندہ پیشانی کے طور سے بہت کم از کم از ہد غور کیا۔ جن جن کی وجہ سے دیکھی جاتی
کے لوگوں میں راوی ضعیف ہی سو رہے گی اور دل یہی حشر ہوتا ہے، اس کے شامل ہوتا ہے تو
ہوں ! لالہ جی، صبح شام کے لوگ مسلط کر رہے تھے۔ دوسرے کو دکھائی یا ایجاد یا یکسوئی، خلوت یا
عجب مربیانہ تبسم ماحول پہ منی آرڈر چلا آتا ہے لیکن ان کتابوں کے سامنے ندامت بتدریج غصے میں ہر ہاسٹل
کہ فلم گزر گیا ہے۔ بات ضرور چاہتا ہے کہ اٹھ کے متعلق کوئی امنگ کوئی گوشہ ایسا جھونکا آئے تو
رہے گا، چنانچے گھر والوں کی کہ ان کی بدولت نہ آئی۔ چنانچے گھر کی کیا میرے جسم کی روحیں اور
اور بعض اوقات میں چند مصوروں کے لئے تو ایک ہی جاگ رہے تھے کہ ان کے گھسیٹنے، کلیاں اور دل
کو فورا یقین نہ آجائیں گے؟ اور کہاں تک قطعی جاگ اٹھی ہوگی۔ e
دیتے اورتعلیم اور استحکام اور عریاں کہہ سکتا تھا۔ سگریٹ سلگا لیتا ہوں ’ نہیں پڑھتا۔ میری مراد یہ بھی اس


## Computing the Probability of Sentences

Compute the probability of each sentence that has been generated in the previous step. Refer to the lecture slides to see what does it mean to compute the _probability of a sentence_. 

Apply the **unigram assumption** while computing these probabilities.


In [22]:
# code here

def find_sent_prob(sentence):
    sent_prob = 1
    sent_list = sentence.split()
    for word in sent_list:
        sent_prob *= tokens.count(word)/token_count
    return sent_prob

sent_prob_list = []
for sentence in paragraph:
    sent_prob = find_sent_prob(sentence=sentence)
    sent_prob_list.append(sent_prob)

        

In [23]:
print(sent_prob_list)


[5.292187227410877e-68, 1.1200115745579694e-64, 2.865866178460349e-58, 4.877948471828708e-64, 3.0370002873339214e-67, 1.9453919993145764e-61, 1.1050536828920059e-58, 2.7465837090388707e-55, 8.481495271018187e-45, 6.000151705764177e-65]


## Discussion and Evaluation

- Analyze the text generated, and mention 3 distinct observations. Also compare it with the input text and how different it is and why might that be.

- Do you notice any repetition of words in the generated sentences? If yes, how would you solve it?

- Is going upto `n=2` enough? What do you think would be a good value of n and why?


Answer here:
<div style="color:green;">
1. 
<li> Even though my code should handle EOS markers (end when a marker encountered), some sentences for a reason unknown to me include the EOS marker
<li> The input text is coherent, meaning, the sentence makes sense to an urdu reader, why the output data does not. This is because the n-gram model is not really 'intelligent' in the sense its just using probailities and also assumes that words to the left and right have no dependence on the word in the middle. 
<li> I guess some sentences are longer than 20 words? 20 being the max cap that i have applied on the sentence length limit. However i can't be sure because its difficult to distinguish whhen a sentence starts and when it ends in urdu. 
</li>
2.
<li> There is a repitition of words in a sentence, but i dont get why i would like to remove it? Its perfectly normal for a sentence to have more than one occurence of a word. There are multiple ways to remove it however, the most intuitive one would be to post-process our sentences to remove any instances more than one of a single word. Another way would be to process the bigram list and removing all the tuples that contain two of the same elements. For example, if [(hello, hello),(hello, i)] -> [(hello, i)]

</li>
3.
<li>There is no one-size-fits-all answer, and the optimal value of n can vary based on the nature of the language and the context of the data. Computational complexity increaes as n goes up, so higher n's are not advised. Higher valules of n can also lead to sparsity issues. Therefore I think n=2 seems a good start but i feel like n=3 is the optimal one, but i cant say for sure before testing it with code

</div>
